# 통합 생명과학 연구 에이전트 - Amazon Bedrock AgentCore

![AgentCore](images/agentcore_overview.png)

이 노트북에서는 모든 노트북(01~04)의 도구들을 통합한 종합적인 생명과학 연구 에이전트를 Amazon Bedrock AgentCore Runtime으로 배포합니다.

## 통합되는 기능
- **외부 데이터베이스** (01_external_dbs): Arxiv, ChEMBL, PubMed, ClinicalTrials
- **내부 데이터베이스** (02_internal_dbs): PostgreSQL 임상/유전체 데이터
- **하이브리드 도구** (03_hybrid_tools): Knowledge Base
- **단백질 설계** (04_protein_design): AWS HealthOmics 워크플로우

## 학습 목표
- 모든 도구를 통합한 종합 Agent 구현
- AgentCore Runtime에 배포 및 운영
- 다양한 생명과학 연구 시나리오 테스트

## 1. 환경 설정

In [ ]:
# 필요한 패키지 설치
%pip install bedrock-agentcore-starter-toolkit strands-agents strands-agents-tools boto3 \
    mcp arxiv chembl-webresource-client python-dateutil pubmedmcp \
    psycopg2-binary --quiet

In [ ]:
# 라이브러리 임포트
import sys
import json
import logging
from typing import Dict, Any

# AWS SDK
import boto3
from boto3.session import Session

# Bedrock AgentCore
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# Strands Agents
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import calculator

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 2. 통합 Agent 구현

모든 노트북의 도구들을 하나의 Agent에 통합한 Python 모듈을 생성합니다.

변경 필요 사항:
1. `DB_CONFIG`의 `host`
2. `trigger_protein_optimization` 함수의 `workflow_id`, `role_arn`, `s3_bucket`
3. `query_knowledge_base` 함수의 `kb_id`

In [ ]:
%%writefile integrated_research_agent.py
"""
통합 생명과학 연구 에이전트 for Amazon Bedrock AgentCore
모든 도구 통합: 외부DB, 내부DB, 하이브리드, 단백질 설계
"""
import json
import logging
import os
from typing import Dict, Any, List, Optional
from collections import defaultdict
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands import Agent, tool
from strands.models import BedrockModel

# 외부 라이브러리
import arxiv
from chembl_webresource_client.new_client import new_client as chembl_client
import httpx
from defusedxml import ElementTree as ET
import psycopg2
import boto3

import uuid

app = BedrockAgentCoreApp()
logger = logging.getLogger(__name__)

# AWS 클라이언트
omics_client = boto3.client('omics')
s3_client = boto3.client('s3')
bedrock_client = boto3.client('bedrock-runtime')
knowledge_base_client = boto3.client('bedrock-agent-runtime')

# =========================
# 1. 외부 데이터베이스 도구
# =========================

@tool
def search_arxiv(query: str, max_results: int = 5) -> List[Dict[str, Any]]:
    """Arxiv에서 학술 논문을 검색합니다
    
    Args:
        query: 검색 쿼리
        max_results: 최대 결과 수 (기본값: 5)
    
    Returns:
        논문 정보 리스트
    """
    try:
        client = arxiv.Client()
        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.SubmittedDate
        )
        
        results = []
        for paper in client.results(search):
            results.append({
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "abstract": paper.summary[:500],
                "url": paper.pdf_url,
                "published": paper.published.isoformat()
            })
        return results
    except Exception as e:
        logger.error(f"Arxiv 검색 오류: {e}")
        return [{"error": str(e)}]

@tool
def search_compound(compound_name: str) -> Dict[str, Any]:
    """ChEMBL에서 화합물 정보를 검색합니다
    
    Args:
        compound_name: 화합물명
    
    Returns:
        화합물 정보 및 활성 데이터
    """
    try:
        molecule = chembl_client.molecule.filter(
            pref_name__iexact=compound_name
        ).only(['molecule_chembl_id', 'pref_name', 'max_phase'])
        
        if molecule:
            mol_data = molecule[0]
            # IC50 활성 데이터
            activity = chembl_client.activity.filter(
                molecule_chembl_id=mol_data['molecule_chembl_id']
            ).filter(standard_type="IC50").only(
                ['pchembl_value', 'assay_description', 'canonical_smiles']
            )[:10]
            
            return {
                "chembl_id": mol_data['molecule_chembl_id'],
                "name": mol_data['pref_name'],
                "max_phase": mol_data.get('max_phase'),
                "activities": list(activity)
            }
        return {"error": "Compound not found"}
    except Exception as e:
        logger.error(f"ChEMBL 검색 오류: {e}")
        return {"error": str(e)}

@tool
def search_pubmed(query: str, max_results: int = 5) -> List[Dict[str, Any]]:
    """PubMed에서 의학 문헌을 검색합니다
    
    Args:
        query: 검색 쿼리
        max_results: 최대 결과 수 (기본값: 5)
    
    Returns:
        논문 정보 리스트
    """
    try:
        base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils"
        
        # 검색
        search_params = {
            "db": "pubmed",
            "term": query,
            "retmax": max_results,
            "retmode": "json"
        }
        
        search_response = httpx.get(
            f"{base_url}/esearch.fcgi",
            params=search_params
        )
        search_data = search_response.json()
        
        id_list = search_data["esearchresult"].get("idlist", [])
        if not id_list:
            return []
        
        # 상세 정보
        fetch_params = {
            "db": "pubmed",
            "id": ",".join(id_list),
            "retmode": "xml"
        }
        
        fetch_response = httpx.get(
            f"{base_url}/efetch.fcgi",
            params=fetch_params
        )
        
        # XML 파싱
        root = ET.fromstring(fetch_response.text)
        articles = []
        
        for article in root.findall(".//PubmedArticle"):
            title = article.find(".//ArticleTitle")
            abstract = article.find(".//AbstractText")
            pmid = article.find(".//PMID")
            year = article.find(".//PubDate/Year")
            
            articles.append({
                "pmid": pmid.text if pmid is not None else "",
                "title": title.text if title is not None else "",
                "abstract": abstract.text[:500] if abstract is not None else "",
                "year": year.text if year is not None else ""
            })
        
        return articles
    except Exception as e:
        logger.error(f"PubMed 검색 오류: {e}")
        return [{"error": str(e)}]

# =========================
# 2. 내부 데이터베이스 도구 (PostgreSQL)
# =========================

# 데이터베이스 설정 - '02_internal_dbs.ipynb' 노트북의 첫번째 cell의 설정으로 동일하게 가져오기
DB_CONFIG = {
    "host": None,
    "port": 5432,
    "database": "agentdb",
    "user": "dbadmin",
    "password": "postgres",
    "connect_timeout": 10
}

@tool
def query_clinical_database(query: str) -> Dict[str, Any]:
    """내부 PostgreSQL 임상 데이터베이스를 쿼리합니다
    
    Args:
        query: SQL 쿼리 또는 자연어 질문
    
    Returns:
        쿼리 결과
    """
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        with conn.cursor() as cursor:
            # SQL 쿼리인지 확인
            if query.strip().upper().startswith(('SELECT', 'WITH')):
                cursor.execute(query)
            else:
                # 자연어를 간단한 SQL로 변환 (실제로는 더 복잡한 NLP 필요)
                cursor.execute(
                    "SELECT table_name FROM information_schema.tables WHERE table_schema='public'"
                )
            
            results = cursor.fetchall()
            return {
                "status": "success",
                "data": results[:100],  # 결과 제한
                "count": len(results)
            }
    except Exception as e:
        logger.error(f"Database 쿼리 오류: {e}")
        return {"error": str(e)}
    finally:
        if 'conn' in locals():
            conn.close()

@tool
def get_database_schema() -> Dict[str, Any]:
    """데이터베이스 스키마 정보를 가져옵니다
    
    Returns:
        테이블 및 컬럼 정보
    """
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        with conn.cursor() as cursor:
            cursor.execute("""
                SELECT
                    c.relname AS table_name,
                    a.attname AS column_name,
                    pg_catalog.format_type(a.atttypid, a.atttypmod) AS data_type
                FROM pg_catalog.pg_attribute a
                JOIN pg_catalog.pg_class c ON a.attrelid = c.oid
                JOIN pg_catalog.pg_namespace n ON c.relnamespace = n.oid
                WHERE a.attnum > 0 AND NOT a.attisdropped
                    AND n.nspname = 'public'
                    AND c.relkind = 'r'
                ORDER BY c.relname, a.attnum;
            """)
            
            rows = cursor.fetchall()
            tables = defaultdict(list)
            for table_name, column_name, data_type in rows:
                tables[table_name].append({
                    "column": column_name,
                    "type": data_type
                })
            
            return {
                "status": "success",
                "schema": dict(tables)
            }
    except Exception as e:
        logger.error(f"스키마 조회 오류: {e}")
        return {"error": str(e)}
    finally:
        if 'conn' in locals():
            conn.close()

# =========================
# 3. 단백질 설계 도구
# =========================

@tool
def trigger_protein_optimization(
    seed_sequence: str,
    runName: Optional[str] = None,
    outputUri: Optional[str] = None,
    esm_model_files: Optional[str] = None,
    onehotcnn_model_files: Optional[str] = None,
    output_type: Optional[str] = None,
    parallel_chains: Optional[str] = None,
    n_steps: Optional[str] = None,
    max_mutations: Optional[str] = None
) -> str:
    """
    Trigger the AWS HealthOmics workflow for protein design optimization
    
    Args:
        seed_sequence: The input protein sequence to optimize
        runName: Name for the workflow run (optional)
        outputUri: S3 URI where the workflow outputs will be stored (optional)
        esm_model_files: S3 directory storing ESM pLM model files (optional)
        onehotcnn_model_files: S3 directory storing Onehot CNN predictor model files (optional)
        output_type: Output type, can be 'best', 'last', or 'all' variants (optional)
        parallel_chains: Number of MCMC chains to run in parallel (optional)
        n_steps: Number of MCMC steps per chain (optional)
        max_mutations: Maximum number of mutations per variant (optional)
    
    Returns:
        String with workflow run information
    """
    try:
        # 계정과 지역 정보 추출
        sts_client = boto3.client('sts')
        account_id = sts_client.get_caller_identity()['Account']
        region = boto3.Session().region_name
        
        # 워크플로우 파라미터 설정 - '04_protein_design_strands.ipynb' 노트북의 6번째 cell에서 출력되는 설정으로 입력
        workflow_id = None
        role_arn = None
        s3_bucket = None
        container_image = f"{account_id}.dkr.ecr.{region}.amazonaws.com/protein-design-evoprotgrad:latest"

        # 서열 검증
        valid_amino_acids = set('ACDEFGHIKLMNPQRSTVWY')
        if not all(aa in valid_amino_acids for aa in seed_sequence.upper()):
            return {"error": "Invalid amino acid seed_sequence"}
                
        run_name = f"workflow-run-{uuid.uuid4().hex[:8]}"
        
        # Use default S3 output location if not provided
        output_uri = f"s3://{s3_bucket}/outputs/{run_name}/"
        
        # Get all workflow parameters with defaults
        esm_files = f"s3://{s3_bucket}/models/esm2_t33_650M_UR50D/"
        out_type = output_type or "all"
        
        # Convert numeric parameters
        try:
            p_chains = int(parallel_chains) if parallel_chains else 10
        except (ValueError, TypeError):
            p_chains = 10
            
        try:
            steps = int(n_steps) if n_steps else 100
        except (ValueError, TypeError):
            steps = 100
            
        try:
            mutations = int(max_mutations) if max_mutations else 15
        except (ValueError, TypeError):
            mutations = 15
        
        # Prepare workflow parameters
        workflow_parameters = {
            "container_image": container_image,
            "seed_sequence": seed_sequence.upper(),
            "esm_model_files": esm_files  # Pass S3 path to workflow
        }
        
        # Add optional parameters
        if onehotcnn_model_files:
            workflow_parameters["onehotcnn_model_files"] = onehotcnn_model_files
            
        workflow_parameters["output_type"] = out_type
        workflow_parameters["parallel_chains"] = p_chains
        workflow_parameters["n_steps"] = steps
        workflow_parameters["max_mutations"] = mutations
        
        # Start the workflow run
        response = omics_client.start_run(
            workflowId=workflow_id,
            name=run_name,
            parameters=workflow_parameters,
            outputUri=output_uri,
            roleArn=role_arn
        )
        
        # Format response
        result = f"""Successfully started protein optimization workflow.

Run ID: {response['id']}
Status: {response['status']}
Output URI: {output_uri}

Optimization parameters:
- Seed sequence: {seed_sequence[:20]}... ({len(seed_sequence)} amino acids)
- Parallel chains: {p_chains}
- Steps per chain: {steps}
- Max mutations: {mutations}
- Output type: {out_type}

You can check the status later by asking me to 'monitor workflow run {response['id']}'"""
        
        return result
        
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def monitor_protein_workflow(runId: str,
    waitForCompletion: Optional[bool] = None,
    pollIntervalSeconds: Optional[int] = None,
    maxWaitTimeMinutes: Optional[int] = None
) -> str:
    """
    Monitor the status of a running AWS HealthOmics workflow and retrieve results when complete
    
    Args:
        runId: The ID of the HealthOmics workflow run to monitor
        waitForCompletion: Whether to wait for the workflow to complete before returning (optional, defaults to True)
        pollIntervalSeconds: Time between status checks in seconds (optional, defaults to 30)
        maxWaitTimeMinutes: Maximum time to wait for workflow completion in minutes (optional, defaults to 30)
    
    Returns:
        String with workflow status and results if completed
    """
    try:
        # Initialize clients
        omics_client = boto3.client('omics')
        s3_client = boto3.client('s3')
        
        if not runId:
            return "Error: runId parameter is required"
        
        # Get workflow run status
        run_response = omics_client.get_run(id=runId)
        status = run_response.get('status')
        
        # Format response with current status
        response_text = f"Run ID: {runId}\nCurrent Status: {status}\n"
        if run_response.get('name'):
            response_text += f"Run Name: {run_response.get('name')}\n"
        if run_response.get('startTime'):
            response_text += f"Start Time: {run_response.get('startTime').isoformat()}\n"
        
        if status == 'COMPLETED':
            # Include results if completed
            response_text += _get_run_results(run_response, s3_client)
        elif status == 'FAILED':
            response_text += f"Failed with message: {run_response.get('statusMessage')}\n"
        else:
            response_text += "\nThe workflow is still running. You can check again later with the same run ID.\n"
            response_text += f"To check again, ask me to 'monitor workflow run {runId}'."
        
        return response_text
        
    except Exception as e:
        return f"Error monitoring workflow: {str(e)}"

def _get_run_results(run_response: Dict[str, Any], s3_client) -> str:
    """
    Helper function to retrieve workflow results from S3.
    """
    try:
        output_text = f"Completed at: {run_response.get('stopTime').isoformat() if run_response.get('stopTime') else 'Unknown'}\n\n"
        
        # Get output files
        output_uri = run_response.get('outputUri')
        if not output_uri:
            return output_text + "No output URI provided."
            
        # Parse the S3 URI
        parsed_uri = urlparse(output_uri)
        bucket = parsed_uri.netloc
        prefix = parsed_uri.path.lstrip('/')
        
        # List objects in the output location
        try:
            s3_response = s3_client.list_objects_v2(
                Bucket=bucket,
                Prefix=prefix
            )
            
            # Add output files to the response
            output_files = []
            for obj in s3_response.get('Contents', []):
                if obj['Key'].endswith('.json') or obj['Key'].endswith('.txt') or obj['Key'].endswith('.csv'):
                    file_key = obj['Key']
                    file_size = obj['Size']
                    
                    # For small text files, include the content
                    if file_size < 10240 and (file_key.endswith('.json') or file_key.endswith('.txt') or file_key.endswith('.csv')):
                        try:
                            file_obj = s3_client.get_object(Bucket=bucket, Key=file_key)
                            file_content = file_obj['Body'].read().decode('utf-8')
                            
                            output_files.append({
                                'key': file_key,
                                'size': file_size,
                                'content': file_content
                            })
                        except Exception as e:
                            output_files.append({
                                'key': file_key,
                                'size': file_size,
                                'error': str(e)
                            })
                    else:
                        output_files.append({
                            'key': file_key,
                            'size': file_size,
                            'url': f"s3://{bucket}/{file_key}"
                        })
            
            if output_files:
                output_text += "Output files:\n"
                for file in output_files:
                    output_text += f"- {file.get('key')} ({file.get('size')} bytes)\n"
                    if 'content' in file and file.get('key').endswith('de_results.csv'):
                        output_text += f"Results summary:\n{file.get('content')[:1000]}...\n\n"
            else:
                output_text += "No output files found."
        except Exception as e:
            output_text += f"Error listing S3 objects: {str(e)}"
            
        return output_text
        
    except Exception as e:
        return f"Error retrieving results: {str(e)}"
        
# =========================
# 4. Knowledge Base 도구
# =========================

@tool
def query_knowledge_base(query: str, kb_id: Optional[str] = None) -> Dict[str, Any]:
    """Amazon Bedrock Knowledge Base를 쿼리합니다
    
    Args:
        query: 검색 쿼리
        kb_id: Knowledge Base ID (선택)
    
    Returns:
        검색 결과
    """
    try:
        # KB 파라미터 설정
        kb_id = None
        if not kb_id:
            return {"error": "Knowledge Base ID not configured"}
        
        response = knowledge_base_client.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={'text': query},
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': 5
                }
            }
        )
        
        results = []
        for item in response.get('retrievalResults', []):
            results.append({
                "content": item['content']['text'],
                "score": item.get('score', 0)
            })
        
        return {
            "status": "success",
            "results": results
        }
    except Exception as e:
        logger.error(f"Knowledge Base 쿼리 오류: {e}")
        return {"error": str(e)}

# =========================
# 메인 Agent 설정
# =========================

# 모델 설정
model_id = "global.anthropic.claude-sonnet-4-5-20250929-v1:0"
model = BedrockModel(model_id=model_id)

# 통합 Agent 생성
agent = Agent(
    model=model,
    tools=[
        # 외부 데이터베이스
        search_arxiv,
        search_compound,
        search_pubmed,
        # 내부 데이터베이스
        query_clinical_database,
        get_database_schema,
        # 단백질 설계
        trigger_protein_optimization,
        monitor_protein_workflow,
        # Knowledge Base
        query_knowledge_base
    ],
    system_prompt="""당신은 종합적인 생명과학 연구 AI 어시스턴트입니다.
    
    사용 가능한 도구:
    1. **외부 데이터베이스**: Arxiv, ChEMBL, PubMed를 통한 문헌 및 화합물 검색
    2. **내부 데이터베이스**: PostgreSQL 임상/유전체 데이터 분석
    3. **단백질 설계**: AWS HealthOmics를 통한 단백질 최적화
    4. **Knowledge Base**: 조직 내부 지식 검색
    
    연구자의 질문에 대해:
    - 적절한 도구를 선택하여 사용
    - 여러 소스의 정보를 종합하여 포괄적인 답변 제공
    - 과학적 정확성 유지
    - 한국어로 친절하게 응답"""
)

# AgentCore 엔트리포인트
@app.entrypoint
def integrated_research_agent(payload: Dict[str, Any]) -> str:
    """AgentCore 엔트리포인트"""
    user_input = payload.get("prompt", "")
    logger.info(f"Processing request: {user_input}")
    
    try:
        response = agent(user_input)
        return response.message['content'][0]['text']
    except Exception as e:
        logger.error(f"Error: {str(e)}")
        return f"오류가 발생했습니다: {str(e)}"

# 실행
if __name__ == "__main__":
    app.run()

## 3. Requirements 파일 생성

In [ ]:
%%writefile requirements.txt
strands-agents>=0.5.0
strands-agents-tools>=0.1.0
boto3>=1.34.0
mcp>=0.1.0
arxiv>=2.1.0
chembl-webresource-client>=0.10.8
python-dateutil>=2.8.2
python-dotenv>=1.0.0
httpx>=0.24.0
defusedxml>=0.7.1
psycopg2-binary>=2.9.0
bedrock-agentcore-starter-toolkit
pubmedmcp

## 4. 환경 변수 설정

In [ ]:
# AWS 세션 설정
boto_session = Session()
region = boto_session.region_name
account_id = boto3.client('sts').get_caller_identity()['Account']

print(f"Region: {region}")
print(f"Account ID: {account_id}")
print(f"환경 변수 설정 완료")

## 5. AgentCore Runtime 구성 및 배포

In [ ]:
# Runtime 인스턴스 생성
agentcore_runtime = Runtime()
agent_name = "integrated_lifescience_research_agent"

# Runtime 설정
response = agentcore_runtime.configure(
    entrypoint="integrated_research_agent.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
print(response)

## 6. Agent 배포

In [ ]:
# AgentCore Runtime에 배포
print("🚀 통합 Agent를 AgentCore Runtime에 배포 중...")
launch_result = agentcore_runtime.launch()

print(f"\n✅ 배포 완료!")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"ECR URI: {launch_result.ecr_uri}")

### 6.1. 정책 복사

In [ ]:
import boto3
import json

iam = boto3.client('iam')
paginator = iam.get_paginator('list_roles')

role_prefix = 'AmazonBedrockAgentCoreSDKRuntime'

# SageMaker Role 이름 가져오기
import sagemaker
sagemaker_role_arn = sagemaker.get_execution_role()
sagemaker_role_name = sagemaker_role_arn.split('/')[-1]

for page in paginator.paginate():
    for role in page['Roles']:
        if role['RoleName'].startswith(role_prefix):
            target_role_name = role['RoleName']
            # 1. 관리형 정책 복사
            attached_policies = iam.list_attached_role_policies(RoleName=sagemaker_role_name)
            for policy in attached_policies['AttachedPolicies']:
                policy_arn = policy['PolicyArn']
                print(f"Attaching managed policy: {policy['PolicyName']}")
                iam.attach_role_policy(
                    RoleName=target_role_name,
                    PolicyArn=policy_arn
                )
            
            # 2. 인라인 정책 복사
            inline_policies = iam.list_role_policies(RoleName=sagemaker_role_name)
            for policy_name in inline_policies['PolicyNames']:
                print(f"Copying inline policy: {policy_name}")
                
                # 정책 문서 가져오기
                policy_doc = iam.get_role_policy(
                    RoleName=sagemaker_role_name,
                    PolicyName=policy_name
                )
                
                # 대상 role에 인라인 정책 추가
                iam.put_role_policy(
                    RoleName=target_role_name,
                    PolicyName=policy_name,
                    PolicyDocument=json.dumps(policy_doc['PolicyDocument'])
                )
            
            print("모든 정책 복사 완료!")

## 7. 배포 상태 확인

In [ ]:
import time

# 배포 상태 확인
print("⏳ Agent 상태 확인 중...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(f"상태: {status}")

print(f"\n✅ 최종 상태: {status}")

## 8. 통합 Agent 테스트

다양한 시나리오로 통합 Agent를 테스트합니다.

### 8.1 외부 데이터베이스 통합 테스트

In [ ]:
# 테스트 1: 문헌 검색 통합
query1 = """COVID-19 백신과 관련된 최신 연구를 조사해주세요. 
Arxiv와 PubMed에서 관련 논문을 찾고, 주요 내용을 요약해주세요."""

print("📚 테스트 1: 문헌 검색 통합")
print(f"질문: {query1}")
print("=" * 60)

response1 = agentcore_runtime.invoke({"prompt": query1})
if 'response' in response1:
    response_text = response1['response'][0]
    if isinstance(response_text, bytes):
        response_text = response_text.decode('utf-8', errors='ignore')
    print(f"응답:\n{response_text}")
print("=" * 60)

### 8.2 내부 데이터 분석 테스트

In [ ]:
# 테스트 2: 화합물 정보
query2 = """데이터베이스에 어떤 테이블들이 있는지 알려주세요."""

print("🧪 테스트 3: 사내 데이터베이스 에이전트 테스트")
print(f"질문: {query2}")
print("=" * 60)

response2 = agentcore_runtime.invoke({"prompt": query2})
if 'response' in response2:
    response_text = response2['response'][0]
    if isinstance(response_text, bytes):
        response_text = response_text.decode('utf-8', errors='ignore')
    print(f"응답:\n{response_text}")
print("=" * 60)

### 8.3 단백질 설계 테스트

In [ ]:
# 테스트 3: 단백질 최적화
test_sequence = "EVQLVETGGGLVQPGGSLRLSCAASGFTLNSYGISWVRQAPGKGPEWVS"
query3 = f"""다음 항체 서열을 최적화해주세요: {test_sequence}
안정성과 결합 친화력을 향상시키는 방향으로 최적화해주세요."""

print("🧬 테스트 3: 단백질 설계")
print(f"질문: {query3}")
print("=" * 60)

response3 = agentcore_runtime.invoke({"prompt": query3})
if 'response' in response3:
  # 바이트 문자열을 UTF-8로 디코딩
  response_text = response3['response'][0]
  if isinstance(response_text, bytes):
      response_text = response_text.decode('utf-8', errors='ignore')
  print(f"응답:\n{response_text}")
print("=" * 60)

### 8.4 복합 연구 질문 테스트

In [ ]:
# 테스트 4: 복합 연구 질문
query4 = """알츠하이머병 치료를 위한 새로운 접근법에 대해 종합적으로 조사해주세요:
1. 최신 연구 논문 (Arxiv, PubMed)
2. 현재 임상시험 중인 약물 (ChEMBL)
3. 우리 데이터베이스의 관련 환자 데이터
4. 타우 단백질 응집 억제를 위한 펩타이드 설계 가능성
"""

print("🔬 테스트 4: 복합 연구 질문")
print(f"질문: {query4}")
print("=" * 60)

response4 = agentcore_runtime.invoke({"prompt": query4})
if 'response' in response4:
    response_text = response4['response'][0]
    if isinstance(response_text, bytes):
        response_text = response_text.decode('utf-8', errors='ignore')
    print(f"응답:\n{response_text}")
print("=" * 60)

## 9. 정리 (Optional)

In [ ]:
# Agent Runtime 삭제 (필요시에만 실행)
# 주의: 이 셀을 실행하면 배포된 Agent가 삭제됩니다!

# cleanup = input("정말로 Agent를 삭제하시겠습니까? (yes/no): ")
# if cleanup.lower() == 'yes':
#     agentcore_control_client = boto3.client(
#         'bedrock-agentcore-control',
#         region_name=region
#     )
#     
#     ecr_client = boto3.client('ecr', region_name=region)
#     
#     try:
#         # Runtime 삭제
#         runtime_delete_response = agentcore_control_client.delete_agent_runtime(
#             agentRuntimeId=launch_result.agent_id
#         )
#         
#         # ECR 리포지토리 삭제
#         ecr_response = ecr_client.delete_repository(
#             repositoryName=launch_result.ecr_uri.split('/')[1],
#             force=True
#         )
#         
#         print("✅ Agent 정리 완료")
#     except Exception as e:
#         print(f"정리 중 오류: {e}")

## 요약

이 노트북에서는 모든 생명과학 연구 도구를 통합한 종합 Agent를 Amazon Bedrock AgentCore Runtime에 성공적으로 배포했습니다.

### 통합된 기능:
- ✅ **외부 데이터베이스**: Arxiv, ChEMBL, PubMed 문헌 및 화합물 검색
- ✅ **내부 데이터베이스**: PostgreSQL 임상/유전체 데이터 분석
- ✅ **단백질 설계**: AWS HealthOmics 워크플로우 기반 최적화
- ✅ **Knowledge Base**: 조직 내부 지식 검색 및 활용

### 주요 성과:
- 모든 도구를 하나의 Agent로 통합
- 서버리스 환경에서 확장 가능한 배포
- 실시간 연구 질문 처리
- 다양한 생명과학 연구 시나리오 지원

### 활용 시나리오:
1. **신약 개발**: 타겟 발굴부터 리드 최적화까지
2. **정밀 의료**: 환자 맞춤형 치료 전략 수립
3. **바이오마커 발굴**: 진단 및 예후 마커 개발
4. **단백질 공학**: 치료용 항체 및 효소 설계
5. **임상 연구**: 데이터 기반 의사결정 지원

### 향후 발전 방안:
1. 추가 외부 데이터 (KEGG, UniProt 등), 내부 데이터 (내부 화합물 정보), 도구 (AlphaFold) 등 통합 
2. 실시간 스트리밍 응답 구현
3. AgentCore의 다른 기능들 (Gateway, Memory, Identity 등) 사용
4. 웹 애플리케이션